## Copyright 2023 Google LLC

In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# StyleAligned over SDXL

In [2]:
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
import mediapy
import sa_handler
import numpy as np
import random
from PIL import Image

def fix_seed(seed=0):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    
def get_concat_h(images):
    height, width = 0, 0
    for image in images:
        width += image.width
        height = max(height, image.height)
    dst = Image.new('RGB', (width, height))
    current_width = 0
    for image in images:
        dst.paste(image, (current_width, 0))
        current_width += image.width
    return dst

/home/jovyan/.mlspace/envs/neudachina/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# init models

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, 
                          beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", use_safetensors=True,
    scheduler=scheduler
).to("cuda")

# handler = sa_handler.Handler(pipeline)
# sa_args = sa_handler.StyleAlignedArgs(share_group_norm=False,
#                                       share_layer_norm=False,
#                                       share_attention=True,
#                                       adain_queries=True,
#                                       adain_keys=True,
#                                       adain_values=False,
#                                      )

# _ = handler.register(sa_args, )

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 12.29it/s]


In [4]:
scheduler = DDIMScheduler(
        beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", 
        clip_sample=False, set_alpha_to_one=False)
    
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    scheduler=scheduler
).to("cuda")


handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(
    share_group_norm=False,
    share_layer_norm=False,
    share_attention=True,
    adain_queries=False,
    adain_keys=False,
    adain_values=False
)

images = []

sa_args.svd = True
sa_args.start_svd = 0
sa_args.end_svd = 5

sa_args.svd_variance_threshold_smallest = 1

_ = handler.register(sa_args, )

Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 10.23it/s]


In [ ]:
sets_of_prompts = [
  "a toy train. macro photo. 3d game asset",
  "a toy airplane. macro photo. 3d game asset"
]
get_concat_h(pipeline(sets_of_prompts, num_inference_steps=20, handler=pipeline).images)

In [3]:
import numpy as np
np.linspace(0.1, 0.6, 11, endpoint=True)

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ])

In [ ]:
fix_seed()
get_concat_h(pipeline(['a blonde girl with blue eyes', 'a blonde girl with blue eyes in a car'], num_inference_steps=20).images)

In [ ]:
# run StyleAligned

sets_of_prompts = [
  "a toy train. macro photo. 3d game asset",
  "a toy airplane. macro photo. 3d game asset",
  "a toy bicycle. macro photo. 3d game asset",
  "a toy car. macro photo. 3d game asset",
  "a toy boat. macro photo. 3d game asset",
]
images = pipeline(sets_of_prompts,).images
mediapy.show_images(images)